# Интеграция. Итоговый проект

Реализация rest api на базе flask в google colab.

В работу добавили колонку 'requirements', изменили классификатор на RandomForest, сделали feature engineering в цикле. Поменяли настройки сервера, добавили колонку 'requirements', запустили сервер. Сделали пробное предсказание. Затем сделали запуск расчета N запросов. Для удобства объединили все этапы в один ноутбук. Но запускать request нужно в отдельном ноутбуке.

# Step 1 - TRAIN

### Обучение пайплайна

1. Загрузим данные https://www.kaggle.com/shivamb/real-or-fake-fake-jobposting-prediction
2. Соберем пайплайн с простейшим препроцессингом (tfidf) на текстовых данных
3. Обучим логистическую регрессию и сохраним на диск предобученный пайплайн

In [49]:
#!pip install dill

In [130]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score

#working with text
from sklearn.feature_extraction.text import TfidfVectorizer

#normalizing data
from sklearn.preprocessing import StandardScaler

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score

#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

In [131]:
!wget 'https://drive.google.com/uc?export=download&id=1T0V44fGNCbgaWh78gZ2ieeoqo5eBowVi' -O fake_job_postings.csv

In [132]:
df = pd.read_csv("./fake_job_postings.csv")
df.head(3)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0


In [133]:
df['fraudulent'].value_counts()

0    17014
1      866
Name: fraudulent, dtype: int64

Разделим данные на train/test и сохраним тестовую выборку на диск

In [134]:
X_train, X_test, y_train, y_test = train_test_split(df, df['fraudulent'],
                                                    test_size=0.33, random_state=42)
# save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)

# save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [135]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    

class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [136]:
features = ['description', 'company_profile', 'benefits', 'requirements']
target = 'fraudulent'

Соберем кусок, ответственный за feature engineering

In [137]:
# combine
transformer_columns_list = []
for column in features:
    transformer = Pipeline([
                ('imputer', TextImputer(column, '')),
                ('selector', ColumnSelector(key=column)),
                ('tfidf', TfidfVectorizer())
            ])
    
    transformer_columns_list.append((column, transformer))

feats = FeatureUnion(transformer_columns_list)

Добавим классификатор random forest

In [138]:
%%time

pipeline = Pipeline([
    ('features', feats),
    ('classifier', RandomForestClassifier(random_state = 0))
])

pipeline.fit(X_train, y_train)

CPU times: user 6.65 s, sys: 19.2 ms, total: 6.67 s
Wall time: 6.67 s


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('description',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='description',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='description')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())])),
                                                ('company_profile',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='company_profile',
                                                     

Посмотрим, как выглядит наш pipeline

In [139]:
pipeline.steps

[('features',
  FeatureUnion(transformer_list=[('description',
                                  Pipeline(steps=[('imputer',
                                                   TextImputer(key='description',
                                                               value='')),
                                                  ('selector',
                                                   ColumnSelector(key='description')),
                                                  ('tfidf', TfidfVectorizer())])),
                                 ('company_profile',
                                  Pipeline(steps=[('imputer',
                                                   TextImputer(key='company_profile',
                                                               value='')),
                                                  ('selector',
                                                   ColumnSelector(key='company_profile')),
                                                  ('tfi

Сохраним модель (пайплайн)

In [140]:
with open("logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

# Step 2 - PREDICT

### Проверка работоспособности и качества пайплайна

Здесь мы еще не запускаем никакое API, а загружаем модель (pipeline) напрямую и проверяем на отложенной (тестовой) выборке

In [141]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [142]:
X_test.head(3)

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,4709,Python Engineer,"GB, , London",NaN,NaN,NaN,Stylect is a dynamic startup that helps helps ...,We don’t care where you studied or what your G...,We are negotiable on salary and there is the p...,0,1,0,Full-time,Entry level,Unspecified,Apparel & Fashion,Information Technology,0
1,11080,Entry Level Sales,"US, OH, Cincinnati",NaN,55000-75000,NaN,General Summary: Achieves maximum sales profit...,NaN,Great Health and DentalFast Advancement Opport...,1,0,0,Full-time,Entry level,High School or equivalent,Financial Services,Sales,0
2,12358,Agile Project Manager,"US, NY, New York",NaN,NaN,ustwo offers you the opportunity to be yoursel...,"At ustwo™ you get to be yourself, whilst deliv...",Skills• Experience interfacing directly with c...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0


In [143]:
with open('logreg_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [144]:
pipeline

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('description',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='description',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='description')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())])),
                                                ('company_profile',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='company_profile',
                                                     

In [145]:
preds = pipeline.predict_proba(X_test)[:, 1]

pred_df = pd.DataFrame({'preds': preds})
pred_df.to_csv("test_predictions.csv", index=None)

In [146]:
preds[:10]

array([0.04, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])

In [147]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.23, F-Score=0.870, Precision=0.960, Recall=0.796


# Step 3 - FLASK

## При внедрении

**При внедрении необходимо сделать:**
*   Определить формат json'а, в котором данные будут приниматься сервисом и отправляться обратно.
*   Определить ip-адрес и порт, на который будут поступать данные.
*   Создать во Flask необходимые роуты:<br/>
    `@app.route('/predict_example', method='POST')`<br/>
    `def predict_example():`
*   Перенести во Flask все функции преобразования данных,
    *   формат данные, приходящих от фронт-системы, может отличаться от формата исторических данных, использовавшихся при построении модели; в результате преобразований данные на вход модели должны поступить ровно в том виде, в каком была обучена модель.
*   Загрузить обученные модели.
*   Настроить логирование.

## Flask

Тут будет сервис для обработки запросов на Flask

In [148]:
#!pip3 install flask-ngrok

In [149]:
#from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import pandas as pd

In [150]:
#!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
#!tar -xvf /content/ngrok-stable-linux-amd64.tgz
#!./ngrok authtoken 25vEpcJ5Ih4vlUp4thEZ9sEA6ZU_3Bnu17gKacRXhF6hLeefc
#!./ngrok http 80

In [151]:
# # Пробный запуск Flask

# app = Flask(__name__)
# #run_with_ngrok(app)  # Start ngrok when app is run

# @app.route("/a")
# def hello():
#     return "Hello World!"

# if __name__ == '__main__':
#     app.run(port=8082)

In [152]:
import pandas as pd
import dill

### **Создаем сервис для обработки запросов к модели**

In [153]:
# Загружаем обученные модели
with open('logreg_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [154]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

Запустить сервис и не глушить его, пока работаем 

In [155]:
# Обработчики и запуск Flask
app = Flask(__name__)
#run_with_ngrok(app)  # Start ngrok when app is run


@app.route("/", methods=["GET"])
def general():
    return "Welcome to prediction process"

@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    description, company_profile, benefits, requirements = "", "", "", ""
    
    request_json = request.get_json()
    
    if request_json["description"]:
        description = request_json['description']
    
    if request_json["company_profile"]:
        company_profile = request_json['company_profile']
                
    if request_json["benefits"]:
        benefits = request_json['benefits']
        
    if request_json["requirements"]:
        requirements = request_json['requirements']
    
    print(description)  
    preds = model.predict_proba(pd.DataFrame({"description": [description],
                                              "company_profile": [company_profile],
                                              "benefits": [benefits],
                                             "requirements": [requirements]}))
    data["predictions"] = preds[:, 1][0]
    data["description"] = description
        # indicate that the request was a success
    data["success"] = True
    print('OK')

        # return the data dictionary as a JSON response
    return jsonify(data)


if __name__ == '__main__':
    app.run(port=8082)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8082/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Jul/2023 13:36:27] "POST //predict HTTP/1.1" 200 -


Stylect is a dynamic startup that helps helps women discover and buy shoes. We’re a small team based in London that has previously worked at Google, Techstars, Pixelmator and Rocket Internet.We place a high premium on simplicity no matter what we’re working on (i.e. design, programming, marketing). We’re also a team that ships fast. We built version 1 of our app in a week, the next release (built in a month) was featured in the Apple Appstore Italy as a best new fashion app. Fast release cycles are challenging, but also very fun - which is why we love them. As we’ve grown, the projects that we’re working on have grown both in scale and in technical complexity.  Stylect is looking for someone who can help us improve our backend which gathers product data; analyses/categorizes it; and shows it to thousands of users daily. Each step in the process has unique challenges that demands a strong technical background.
OK


127.0.0.1 - - [19/Jul/2023 13:37:43] "POST //predict HTTP/1.1" 200 -


Stylect is a dynamic startup that helps helps women discover and buy shoes. We’re a small team based in London that has previously worked at Google, Techstars, Pixelmator and Rocket Internet.We place a high premium on simplicity no matter what we’re working on (i.e. design, programming, marketing). We’re also a team that ships fast. We built version 1 of our app in a week, the next release (built in a month) was featured in the Apple Appstore Italy as a best new fashion app. Fast release cycles are challenging, but also very fun - which is why we love them. As we’ve grown, the projects that we’re working on have grown both in scale and in technical complexity.  Stylect is looking for someone who can help us improve our backend which gathers product data; analyses/categorizes it; and shows it to thousands of users daily. Each step in the process has unique challenges that demands a strong technical background. 
OK


...

OK
We are seeking a Customer Service Representative who enjoys building relationships, helping others, solving problems, and organizing a demanding schedule. The ideal candidate is independent, confident, and customer focused with good time management skills, strong communication skills and excellent computer aptitude that can help resolve customer problems both internally and externally.
OK
Network Closing Services, Inc., a full service Title Company is seeking an established Sales Representative with a Title Insurance Book of Business.  We are continuing our nationwide expansion.  Come join a winning team!Network Closing Services has been serving Lenders, Real Estate Consumers, and Professionals since 1999.  We provide courteous professional services, speedy title searches, and timely disbursements.  Dynamic flexibility is key to our success.  Our Client satisfaction is very important.  We provide experienced settlement officers and attorneys anywhere in the country, day or night, to

127.0.0.1 - - [19/Jul/2023 13:58:10] "POST //predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 13:58:10] "POST //predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 13:58:10] "POST //predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 13:58:10] "POST //predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 13:58:10] "POST //predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 13:58:10] "POST //predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 13:58:10] "POST //predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 13:58:10] "POST //predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 13:58:10] "POST //predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 13:58:10] "POST //predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 13:58:10] "POST //predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 13:58:10] "POST //predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 13:58:10] "POST //predict HTTP/1.1" 200 -
127.0.0.1 - - [19/Jul/2023 13:58:10] "POST //predict HTTP/1.1" 200 -


OK
Our company is looking for candidates for the position of Developer.Rensposibilities of successful candidates include:Owning and investigating a wide variety of issues at both an application and database levelProducing robust, well written solutionsDeployment of fixes to both test &amp; live environments, depending on defined Operator processEffectively communicating investigation status to the OperatorEscalating issues appropriately to ensure quick resolutionLiaising with the development and problem management teams as appropriate to resolve issues
OK
About StageBlocStageBloc is a platform that helps you better understand &amp; connect with your audience and own your digital relationships.Though simple to use, StageBloc gives you the power to incorporate everything you are doing into a single experience.By merging multiple tools &amp; services together, we help you create &amp; maintain incredible digital experiences, understand &amp; connect with your fans in meaningful, measurabl

127.0.0.1 - - [19/Jul/2023 13:58:10] "POST //predict HTTP/1.1" 200 -


OK


# Step 4 - REQUEST (следует выполнять в отдельном ноутбуке)

Строим запрос к нашему сервису

In [ ]:
import requests

import urllib.request
import json 

In [ ]:
# Пример данных
data = ( 
    "Stylect is a dynamic startup that helps helps women discover and buy shoes. We’re a small team based in London that has previously worked at Google, Techstars, Pixelmator and Rocket Internet.We place a high premium on simplicity no matter what we’re working on (i.e. design, programming, marketing). We’re also a team that ships fast. We built version 1 of our app in a week, the next release (built in a month) was featured in the Apple Appstore Italy as a best new fashion app. Fast release cycles are challenging, but also very fun - which is why we love them.\xa0As we’ve grown, the projects that we’re working on have grown both in scale and in technical complexity. \xa0Stylect is looking for someone who can help us improve our backend which gathers product data; analyses/categorizes it; and shows it to thousands of users daily. Each step in the process has unique challenges that demands a strong technical background.",
    "ustwo offers you the opportunity to be yourself, whilst delivering the best work on the planet for some of the biggest and most innovative brands. A culture thriving on collaboration underpins what is an amazing work smart/ live well environment.We genuinely care about the work that we deliver and the people who help make it all possible. We only invest in projects, people and practices that we believe in, to ensure we remain excited about every opportunity.",
    "We are negotiable on salary and there is the potential for equity for the right candidate.",
    "We don’t care where you studied or what your GPA in college was. But we do want someone who is:Strong with Python and No SQL DatabasesEntrepreneurial--independent and can solve problems without being micro-managedCan brainstorm multiple approaches to solving problems instead of being attached to just one way of doing thingsIcing on the cakePrevious experience with Google Appengine’s Datastore (NDB)Previous experience at e-commerce and app-based startupsFull Stack Development Experience from the Frontend (either with apps or html) all the way to the linux server.An understanding of algorithms and experience with libraries to implement themExperience with machine learning and natural language processing"
)

In [ ]:
# формируем запрос
def send_json(x):
    description, company_profile, benefits, requirements = x
    #print(description, company_profile, benefits)
    body = {
        'description': description, 
        'company_profile': company_profile,
        'benefits': benefits,
        'requirements': requirements
        }
    #myurl = 'http://948a-35-230-58-89.ngrok.io/' + '/predict'
    myurl = 'http://127.0.0.1:8082/' + '/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

In [ ]:
# обращение к серверу с запросом из одного набора (его построили руками выше - data)
response = send_json(data)
print('предсказание', response)

Сделаем обработку массы запросов:

загрузим данные X_test

In [ ]:
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
from urllib import request, parse


X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

X_test.fillna('', inplace=True)

In [ ]:
X_test[['description', 'company_profile', 'benefits', 'requirements']]

In [ ]:
Передаем запрос одиночный из наших данных и посмотрим на ответ

In [ ]:
response = send_json(X_test[['description', 'company_profile', 'benefits', 'requirements']].iloc[0,:])

In [ ]:
response

In [ ]:
Сделаем N запросов и оценим время

In [ ]:
N = 500

In [ ]:
%%time
predictions = X_test[['description', 'company_profile', 'benefits', 'requirements']].iloc[:N].apply(lambda x: send_json(x), axis=1)

In [ ]:
predictions.values[:5]

Посчитаем метрику

In [ ]:
import numpy as np

precision, recall, thresholds = precision_recall_curve(y_test[:N], predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

In [ ]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

Наша модель randomforest работает хорошо, на N=500 тестовых данных мы получили метрики:

Best Threshold=0.19, F-Score=0.923, Precision=1.000, Recall=0.857, roc_auc_score=0.991698976041356